This notebook uses the OpenStudio Meta-CLI to submit OSA jobs to OpenStudio-Server.  

The OpenStudio Meta-CLI is a ruby executable that can parse OpenStudio Analysis (OSA) json files and submit the appropriate job type and payload to a running OpenStudio-Server. These files can describe complex analyses, such as building-stock analysis, design optimization, uncertainty quantification, and calibration.  This example requires a running server.

The variables below will use the paths that are appropriate for running this notebook in the provided Docker container.  To use it outside, just change the commented out variables below and change to the appropriate file locations for your specific system.

If this is running in the provided docker container, to get the HOST IP address, in a terminal or command p use  
`>docker info`  
and look for and use the **Node Address: xx.xx.xx.xx**

In [1]:
RUBY_CMD = 'ruby'
#RUBY_CMD = 'C:\ParametricAnalysisTool-3.3.0\pat\ruby\bin\ruby'
META_CLI = File.absolute_path('/opt/OpenStudio-server/bin/openstudio_meta')
#META_CLI = File.absolute_path('C:\ParametricAnalysisTool-3.3.0\pat\OpenStudio-server\bin\openstudio_meta')
PROJECT = Dir.pwd
HOST = '192.168.65.3'

@host = HOST
@project = PROJECT
@meta_cli = META_CLI
@ruby_cmd = RUBY_CMD

"C:\\ParametricAnalysisTool-3.3.0\\pat\\ruby\\bin\\ruby"

Command to run analysis.  Can add the --debug --verbose flags for more info if there are issues.

The location of the .zip file is given by the -z argument.  This can be left out if the name of the zip file is the same as the OSA json.  
The analysis type is given by the -a argument, in this case it is **Morris**  
The http string argument is the IP address of the OS-Server instance.

In [2]:
command = "#{@ruby_cmd} #{@meta_cli} run_analysis '#{@project}/analysis.json' 'http://#{@host}' -z 'analysis' -a morris"
#command = "#{@ruby_cmd} #{@meta_cli} run_analysis --debug --verbose '#{@project}/analysis.json' 'http://#{@host}' -z 'analysis' -a morris"

"C:\\ParametricAnalysisTool-3.3.0\\pat\\ruby\\bin\\ruby C:/ParametricAnalysisTool-3.3.0/pat/OpenStudio-server/bin/openstudio_meta run_analysis 'C:/Projects/Notebooks/osw_project/analysis.json' 'http://localhost' -z 'analysis' -a lhs"

A System call can be used to run analysis, however the output capabilities are not the best

In [3]:
#run_analysis = system(command)

Use open3 to make the system call since it has better stdout capabilities.  This is essentially the command that the PAT GUI uses to submit an Analysis job to an OpenStudio-Server Stack.

In [4]:
require 'open3'

# Make a system call to get version
stdout, stderr, status = Open3.capture3("#{command}")

# Check the result
if status.success?
  puts "Command executed successfully"
  puts stdout
else
  puts "Command failed"
  puts stderr
end

Command executed successfully
OpenStudio did not load, but most functionality is still available. Will continue...
new project created with ID: 85d0687e-9633-4af7-ad05-132587ca27f7
asked to create analysis with 93dcbe9a-931a-4a3d-9357-ba031599c77e
options[:push_to_dencity] = false
new analysis created with ID: 93dcbe9a-931a-4a3d-9357-ba031599c77e
Successfully uploaded ZIP file
Run analysis is configured with {"analysis_action":"start","without_delay":false,"analysis_type":"lhs","simulate_data_point_filename":"simulate_data_point.rb","run_data_point_filename":"run_openstudio_workflow_monthly.rb"}
Received request to run analysis 93dcbe9a-931a-4a3d-9357-ba031599c77e
Run analysis is configured with {"analysis_action":"start","without_delay":false,"analysis_type":"batch_run","simulate_data_point_filename":"simulate_data_point.rb","run_data_point_filename":"run_openstudio_workflow_monthly.rb"}
Received request to run analysis 93dcbe9a-931a-4a3d-9357-ba031599c77e


sleep to let analysis process

In [5]:
sleep 5

5

From this point on, we will demonstrate some scripting / code that will use the Server API to get the submitted analyses and check the status of the running datapoints.  This is not necessary since the server already has the submitted job from the previous step.

Get the UUIDs of the Analyses on the Server and sort by the time to get the one we just created.

In [6]:
require 'rest-client'
require 'json'
a = RestClient.get "http://#{@host}/analyses.json"
a = JSON.parse(a, symbolize_names: true)
a = a.sort { |x, y| x[:created_at] <=> y[:created_at] }.reverse
analysis = a[0]
analysis_id = analysis[:_id]

"93dcbe9a-931a-4a3d-9357-ba031599c77e"

get the status of the Analysis

In [7]:
a = RestClient.get "http://#{@host}/analyses/#{analysis_id}/status.json"
a = JSON.parse(a, symbolize_names: true)
status = a[:analysis][:status]

"started"

get the analysis type

In [8]:
a = RestClient.get "http://#{@host}/analyses/#{analysis_id}.json"
a = JSON.parse(a, symbolize_names: true)
status = a[:analysis][:problem][:analysis_type]

"lhs"

get the current datapoints from the analysis, check their status and loop until the 30 datapoints are complete
This is okay for a small problem.

In [9]:
status = 'queued'
timeout_seconds = 360
begin
  ::Timeout.timeout(timeout_seconds) do
    while status != 'completed'
      # get the analysis pages
      get_count = 0
      get_count_max = 50
      begin
        a = RestClient.get "http://#{@host}/analyses/#{analysis_id}/status.json"
        a = JSON.parse(a, symbolize_names: true)
        analysis_type = a[:analysis][:analysis_type]
        status = a[:analysis][:status]
        puts "Accessed pages for analysis: #{analysis_id}, analysis_type: #{analysis_type}, status: #{status}"
        jobs = a[:analysis][:jobs]
        puts "jobs: #{jobs}"

        a = RestClient.get "http://#{@host}/analyses/#{analysis_id}.json"
        a = JSON.parse(a, symbolize_names: true)
        status_message = a[:analysis][:status_message]
        puts "status_message: #{status_message}"
        # get all data points in this analysis
        a = RestClient.get "http://#{@host}/data_points.json"
        a = JSON.parse(a, symbolize_names: true)
        data_points = []
        a.each do |data_point|
          if data_point[:analysis_id] == analysis_id
            data_points << data_point
          end
        end
        # confirm that queueing is working
        data_points.each do |data_point|
          # get the datapoint pages
          data_point_id = data_point[:_id]
          a = RestClient.get "http://#{@host}/data_points/#{data_point_id}.json"
          a = JSON.parse(a, symbolize_names: true)
          data_points_status = a[:data_point][:status]
          puts "Accessed pages for data_point #{data_point_id}, data_points_status = #{data_points_status}"
        end
      rescue RestClient::ExceptionWithResponse => e
        puts "rescue: #{e} get_count: #{get_count}"
        sleep Random.new.rand(1.0..10.0)
        retry if get_count <= get_count_max
      end
      puts ''
      sleep 10
    end
end
rescue ::Timeout::Error
    puts "Analysis status is `#{status}` after #{timeout_seconds} seconds; assuming error."
end

Accessed pages for analysis: 93dcbe9a-931a-4a3d-9357-ba031599c77e, analysis_type: batch_run, status: started
jobs: [{:index=>0, :analysis_type=>"lhs", :status=>"completed", :status_message=>""}, {:index=>1, :analysis_type=>"batch_run", :status=>"started", :status_message=>""}]
status_message: 
Accessed pages for data_point 043871d6-af27-4599-84c4-11fea314accb, data_points_status = queued
Accessed pages for data_point 804f70f4-4843-4e30-a502-15fcfba032ec, data_points_status = queued
Accessed pages for data_point 5b3407c3-177c-4513-b45d-4988a67b861a, data_points_status = queued
Accessed pages for data_point f7d904bd-68f7-4aa6-a505-280613e48598, data_points_status = queued
Accessed pages for data_point 1b69bbc4-71c0-41bf-817e-dc39e42ca5a0, data_points_status = queued
Accessed pages for data_point b3ad3df5-fee3-448e-88cf-6e11af4ad409, data_points_status = queued
Accessed pages for data_point 5b000f56-f3e3-4d39-9974-ddefa4562101, data_points_status = queued
Accessed pages for data_point 39f

Use the API to download the Results CSV for the analysis and save it as 'results.csv'

In [10]:
require 'open-uri'
File.write 'results.csv', URI.open("http://#{@host}/analyses/#{analysis_id}/download_data.csv?export=true").read


2677

Morris method computes graphical and json results and can be downloaded using the following API

In [ ]:
a = RestClient.get "http://#{@host}/analyses/#{analysis_id}/download_algorithm_results_zip"
puts a.headers[:status]
puts a.headers[:content_type]

#this will make a zip file containing the zipped results.
File.open('results.zip', 'wb') do |file|
   file.write(a.body) 
end

195186